## Ensembles of Decision Trees

*Ensembles* are methods that combine multiple machine learning models to create more powerful models.  *Random forests* and *gradient-boosted decision trees* are two ensemble models that have proven to be effective on a wide range of datasets for classification and regression.  Both use decision trees as their building blocks.

### Random Forests

A main drawback of decision trees is that they tend to overfit the training data.  Random forests are one way to address this problem.  

A *random forest* is essentially a collection of decision trees, where each tree is slightly different from the others.  The idea behind random forests is that each tree might do a relatively good job of predicting, but will likely overfit on part of the data.  If we build many trees, all of which work well and overfit in different ways, we can reduce the amount of overfitting by averaging their results.  This reduction of overfitting can be shown using rigorous mathematics.

To implement this strategy, we need to build many decision trees.  Each tree should do an acceptable job of predicting the target, and should be different from the other trees.

Random forests get their name from injecting randomness into the tree building to ensure each tree is different.  There are 2 ways in which the trees in a random forest are randomized:
- By selecting the data points used to build a tree
- By selecting the features in each test split

### Building Random Forests

To build a random forest model, you need to decide on the number of trees to build (the *n_estimators* parameter of *RandomForestRegressor* or *RandomForestClassifier*).  Let's say we want to build 10 trees.  These trees will be completely independent of each other, and the algorithm will make different random choices for each tree to make sure the trees are distinct.

To build a tree, we first take what is called a *bootstrap sample* of our data.  That is, from our *n_samples* data points, we repeatedly draw an example randomly with replacement, *n_samples* times.  This will create a dataset that is as big as the original dataset, but some data points will be missing from it (approximately 1/3) and some will be repeated

Next, a decision tree is built based on this newly created dataset.  However, the algorithm we described for the decision tree is slightly modified.  Instead of looking for the best test for each node, in each node the algorithm randomly selects a subset of features, and it looks for the best possible test involving one of these features.  The number of features that are selected is controlled by the *max_features* parameter.  This selection of a subset of features is repeated separately in each node, so that each node in a tree can make a decision using a different subset of the features.

The bootstrap sampling leads to each decision tree in the random forest being built on a slightly different dataset.  Because of the selection of features in each node, each split in each tree operates on a different subset of features.  Together, these two mechanisms ensure that all the trees in the random forest are different.

The selection of *max_features* is critical.  If we set *max_features* to *n_features*, that means each split can look at all features in the dataset, and no randomness will be injected into the feature selection.  This means the trees in the forest will be quite similar.  If we set *max_features* to 1, that means the splits have no choice at all on which features to test, and can only search over different thresholds for the feature that was selected randomly.  This means the trees in the forest will be quite different.

To make a prediction using the random forest, the algorithm first makes a prediction for every tree in the forest.  For regression, we can average these results to get our final prediction.  

For classification, a *soft voting* strategy is used.  Each algorithm makes a soft prescription, providing a probability for each possible output label.  The probabilities predicted by all the trees are averaged, and the class with the highest probability is predicted.

### Analyzing Random Forests

Let's apply a random forest consisting of 5 trees to the *two_moons* dataset we studied earlier.

In [2]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import sklearn
from IPython.display import display
import mglearn

# Don't display deprecation warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=100, noise=0.25, random_state=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
forest = RandomForestClassifier